# 第3回 その3: 相関係数と相互相関関数
 


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/03"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/03/
# 現在のフォルダの中身を表示
%ls

`icecream.csv` と`sun.csv`というデータが表示されていることを確認してください。

## ステップ1: 相関係数
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`icecream.csv` を読み込み，データをプロットします。  
このデータは，2017年から2019年までの，東京の月別平均気温と，一世帯あたりのアイスクリームの平均支出金額を記録したデータです。  
(出典：気象庁HPおよび政府統計の総合窓口e-stat)

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('icecream.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) として抽出する。
kion = csv_data.loc[:, '平均気温(℃)'].to_numpy()
ice = csv_data.loc[:, 'アイス支出額(円)'].to_numpy()

# 抽出したデータをプロット
plt.figure(figsize=(10,8))
x = np.arange(np.size(kion))
# 2分割した上段に気温をプロット
plt.subplot(2,1,1)
plt.plot(x, kion, color='b')
plt.xlabel('Month')
plt.ylabel('Temperature [degree]')
# 下段にアイス支出額をプロット
plt.subplot(2,1,2)
plt.plot(x, ice, color='b')
plt.xlabel('Month')
plt.ylabel('Expending for icecream [yen]')
plt.show()

# また，横軸を気温，縦軸をアイス支出額として散布図をプロット
plt.figure(figsize=(5,5))
plt.scatter(kion, ice, color='b')
plt.xlabel('Temperature [degree]')
plt.ylabel('Expending for icecream [yen]')
plt.show()

上の図から，気温とアイスクリームの支出額（売上）には強い関係があることが分かります。  
これら2変数の相関係数を計算してみましょう。  
相関係数を求める関数を以下に定義します。

In [ ]:
# 相関係数を求める関数
def calc_corr_coef(x, y):
  '''
      x, y: 相関を求めたいデータ
  '''
  mu_x = np.mean(x)
  mu_y = np.mean(y)
  std_x = np.std(x)
  std_y = np.std(y)

  corr_coef = np.mean((x - mu_x) * (y - mu_y)) / (std_x * std_y)

  return corr_coef

定義した `calc_correlation` 関数を用いて，気温とアイスクリーム支出額の相関係数を求めます。

In [ ]:
corr_coef = calc_corr_coef(kion, ice)

print('correlation coefficent = %f' % (corr_coef))

相関係数が 1 に近いことから，これら二つの変数には強い相関があることが分かります。

ちなみに，相関係数を求める関数は numpy や pandas にも用意されています。

In [ ]:
print(np.corrcoef(kion, ice))
print(csv_data.corr())

## ステップ2: 相互相関関数

`sun.csv` を読み込み，データをプロットします。  
このデータは，3日間の時間毎の気温と太陽高度を示したデータです。  

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('sun.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) として抽出する。
kion = csv_data.loc[:, '気温(℃)'].to_numpy()
sun = csv_data.loc[:, '太陽高度(degree)'].to_numpy()

# 抽出したデータをプロット
plt.figure(figsize=(10,8))
x = np.arange(np.size(kion))
# 2分割した上段に気温をプロット
plt.subplot(2,1,1)
plt.plot(x, kion, color='b')
plt.xlabel('Time [hour]')
plt.ylabel('Temperature [degree]')
# 下段に太陽高度をプロット
plt.subplot(2,1,2)
plt.plot(x, sun, color='b')
plt.xlabel('Time [hour]')
plt.ylabel('Solar altitude [degree]')
plt.show()

気温と太陽高度も，似た変化をしていることから，高い相関があることが覗えます。  
しかし，よく見ると太陽高度が最高値を取る時間と，気温が最高値を取る時間には少しずれがあるようにも見えます。  

実際に「時間ずれ」があるのか，あった場合は何時間ずれているのか，相互相関関数を使って計算してみましょう。

相互相関関数を求める関数を以下に定義します。  

In [ ]:
# 相互相関関数を求める関数
def calc_cross_corr(x, y):
  '''
      x, y: 相互相関関数を計算するデータ
  '''
  num_samples = np.size(y)
  cross_corr = np.zeros(num_samples)

  for m in range(num_samples):
    # データy を nサンプル分，前へずらす（前へずらした分，後ろにはゼロが入る）
    shifted_y = np.zeros(num_samples)
    shifted_y[:num_samples-m] = y[m:]
    # ずらしたデータと内積を計算
    cross_corr[m] = np.dot(x, shifted_y)
  return cross_corr

定義した `calc_cross_corr`関数を用いて，太陽高度と気温のずれを調べます。  
ここでは，表示上見やすくするために，事前に太陽高度と気温の各データを平均0, 分散1になるように正規化処理しておきます。  

In [ ]:
# 各データから平均値を引き，さらに標準偏差で割ると，平均0，分散1のデータになる（正規化処理）
sun_norm = (sun - np.mean(sun)) / np.std(sun)
kion_norm = (kion - np.mean(kion)) / np.std(kion)

# 相互相関関数を計算する。
corr = calc_cross_corr(sun_norm, kion_norm)
# さらに表示の都合上，corrをサンプル数で割る
corr_norm = corr / np.size(sun)

上記のような正規化処理を行うことで，相互相関関数の値が -1～1 の範囲に収まり，相関係数と同じように考えることができるようになります。

では計算した相互相関関数をプロットするとともに，最大値を取る時間を表示します。

In [ ]:
# (正規化)相互相関関数をプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(kion))
plt.plot(x, corr_norm, color='b')
plt.xlabel('Time [hour]')
plt.ylabel('Normalized Cross Correlation')
plt.show()
 
# 相互相関関数が最大となる時間を得る
max_index = np.argmax(corr_norm)
print('argmax of corr_norm = %d (hours)' % (max_index))
print('max of corr_norm = %f' % (np.max(corr_norm)))

上の結果から，相互相関関数は 時間=2 のときに最大値を取ることが分かりました。  
つまり，太陽高度と気温には約 2 時間のずれがあり，そのずれを補正すると 0.8 程度の相関係数が得られることが分かります。  
実際，太陽は12時に登り切りますが，太陽によって地面が温められ，地面の熱によって気温が上昇するため，最高気温に達するまでには2時間程度の遅延が生じると言われています。  

ちなみに，相互相関関数は14時間のあたりで -0.6程度の逆相関を示しています。  
これは，太陽高度のデータと気温のデータを14時間（2時間のずれ+半日分）ずらすと，真逆の関係になるためです。  

さらに，26時間，50時間の辺りでも相互相関関数にピークがあります。  
これは，2時間のずれ+1日，2日ずらすと，また似たようなデータ同士になるためです。  
言い換えると，太陽高度と気温は24時間周期で似たような値を取るということです。